In [2]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /Users/inflaton/code/engd/papers/DM-Fake-News-Detection


## Create Rewritten Train Set

- `results/llm_rewrite_results_train_data_{i + 1}.csv` (i = 0~3) were created by qwen2.5:7b
- `dataset/rewritten_train_data_{i + 1}.csv` will be created

In [8]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define function to process text
import string
from nltk.stem import *
from nltk.stem.porter import *

# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize

# Part-of-speech tagging
from nltk import pos_tag

# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk

# Regular expressions for text pattern matching
import re


def process_full_review(text):
    # Convert to lowercase and tokenize
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in string.punctuation]
    stemmer = PorterStemmer()
    # List of stopwords
    stop_words = stopwords.words("english")
    allowed_words = [
        "no",
        "not",
        "don't",
        "dont",
        "don",
        "but",
        "however",
        "never",
        "wasn't",
        "wasnt",
        "shouldn't",
        "shouldnt",
        "mustn't",
        "musnt",
    ]

    stemmed = [
        stemmer.stem(word)
        for word in tokens
        if word not in stop_words or word in allowed_words
    ]
    return " ".join(stemmed)

In [10]:
import ast


def process_full_review_for_file(final_file, file_name=None):
    if file_name is None:
        file_name = f"results/llm_rewrite_results_train_data_{final_file}"
    print("loading:", file_name)
    df_rewritten = pd.read_csv(file_name)
    qwen_2_5_7b_result = df_rewritten.columns[-1]
    print("qwen_2_5_7b_result:", qwen_2_5_7b_result)
    df_rewritten["full_content"] = df_rewritten[qwen_2_5_7b_result].apply(
        lambda x: ast.literal_eval(x)["content"]
    )
    df_rewritten.drop(columns=[qwen_2_5_7b_result], inplace=True)

    tqdm.pandas(desc="Processing Train Data")
    df_rewritten["processed_full_content"] = df_rewritten[
        "full_content"
    ].progress_apply(lambda x: pd.Series(process_full_review(x)))
    file_name = f"dataset/rewritten_{final_file}"
    df_rewritten.to_csv(file_name, index=False)
    print("created:", file_name)

In [11]:
process_full_review_for_file(
    "test_data.csv", file_name="results/llm_rewrite_results.csv"
)

loading: results/llm_rewrite_results.csv
qwen_2_5_7b_result: qwen2.5:7b_8k/shots-00(3.319)


Processing Train Data: 100%|██████████| 6050/6050 [00:23<00:00, 257.87it/s]


created: dataset/rewritten_test_data.csv


In [ ]:
for i in range(4):
    process_full_review_for_file(f"train_data_{i + 1}.csv")

loading: results/llm_rewrite_results_train_data_1.csv
qwen_2_5_7b_result: qwen2.5:7b_8k/shots-00(8.691)


Processing Train Data: 100%|██████████| 13611/13611 [01:14<00:00, 183.39it/s]


created: dataset/rewritten_train_data_1.csv
loading: results/llm_rewrite_results_train_data_2.csv
qwen_2_5_7b_result: qwen2.5:7b_8k/shots-00(8.713)


Processing Train Data: 100%|██████████| 13610/13610 [00:33<00:00, 410.96it/s]


created: dataset/rewritten_train_data_2.csv
loading: results/llm_rewrite_results_train_data_3.csv
qwen_2_5_7b_result: qwen2.5:7b_8k/shots-00(8.583)


Processing Train Data: 100%|██████████| 13610/13610 [00:33<00:00, 408.22it/s]


created: dataset/rewritten_train_data_3.csv
loading: results/llm_rewrite_results_train_data_4.csv
qwen_2_5_7b_result: qwen2.5:7b_8k/shots-00(8.667)


Processing Train Data: 100%|██████████| 13610/13610 [00:37<00:00, 367.02it/s]


created: dataset/rewritten_train_data_4.csv


In [41]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "original": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "rewritten": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
    },
)
datasets

Generating original split: 54441 examples [00:02, 26715.39 examples/s]
Generating rewritten split: 54441 examples [00:01, 36274.75 examples/s]


DatasetDict({
    original: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
})

## Rewrite news using DeepSeek-V3

- Prompt to gpt-4o
``` 
rate the below rewritten news agains this original: MEXICO CITY (Reuters) - Donald Trump’s combative style has buffeted Mexico’s president for months, but deeply unpopular Enrique Pena Nieto may end up thanking the new U.S. president for prompting offended Mexicans to rally behind their leader.  From billionaire business magnate Carlos Slim to political opponents, there has been a groundswell of support for Pena Nieto, who has cut a lonely figure in months of bruising encounters with Trump. Often referred to by his initials EPN, Pena Nieto is laboring under the worst approval ratings of any Mexican president in decades due to discontent over corruption, gang violence, sluggish growth and a jump in fuel prices.     Trump’s threats to scrap the NAFTA trade deal with Mexico and build a border wall have caused anger and left Mexicans feeling fearful for the future. His challenge to Pena Nieto on Thursday - saying he should skip a planned summit between the two leaders if Mexico wasn’t willing to pay for the wall - was the final straw. Pena Nieto replied a couple of hours later that he had canceled his meeting with Trump, one of the president’s biggest ever hits on Twitter, getting more “likes” than when he personally broke news of the capture of notorious drug lord Joaquin “El Chapo” Guzman in January of last year. “Bravo EPN!,” cheered former President Vicente Fox, who had initially condemned Pena Nieto for inviting Trump to Mexico for talks last August, and who has waged a colorful and expletive-ridden campaign against the Republican on Twitter. Pena Nieto and Trump talked for about an hour by phone on Friday, pledging to work out their differences and agreeing not to speak about the wall in public for now. Meanwhile, calls for unity grew in Mexico, led by Slim, a normally media-shy 76-year-old who gave a 90-minute news conference in support of the government on Friday. “This is the most surprising example of national unity I’ve seen in my life,” said Slim, who spent several years in the past decade as the world’s richest man. “We have to back the president of Mexico so he can defend our national interests.” Senior opposition leaders also urged a common front. “It’s time to show unity and our commitment to Mexico,” Alejandra Barrales, head of the center-left Party of the Democratic Revolution (PRD), said on Twitter. Pena Nieto’s credibility has been battered by a widespread public perception that he has failed to battle corruption and indeed even encouraged it since a conflict-of-interest row embroiled him, his wife and a top minister in late 2014. Some opposition critics fear the government will try to use Trump as a screen to distract from its failings. However, two senior officials told Reuters they hoped Pena Nieto would seize the moment to act quickly and decisively to improve his image. Expressions of dismay at Trump’s behavior towards Mexico have almost become a national pastime, and talk of boycotts against U.S. companies is gathering steam on social media. Slim, when asked about boycotting at his news conference, said it wasn’t a good idea to turn on U.S. companies, which are creating jobs in Mexico. Some foreign companies voiced support for Mexico too. “I feel Mexico is being subjected to terror at the moment,” said Andreas Schindler, co-owner of German fruit wholesaler Don Limon’s parent Pilz Schindler, by phone from Hamburg. “We’re right behind Mexico.”  Trump tussle gives unpopular Mexican leader much-needed shot in arm

rewritten: **MEXICO CITY (Reuters)** — Mexican President Enrique Peña Nieto, grappling with historic unpopularity, has unexpectedly garnered a wave of national solidarity amid escalating tensions with U.S. President Donald Trump, whose confrontational rhetoric has unified Mexicans in defense of their country’s dignity and interests.  

Peña Nieto, often abbreviated as EPN, has faced public fury over corruption scandals, cartel violence, economic stagnation, and a controversial fuel price surge, leaving his approval ratings at record lows. Yet Trump’s persistent threats—from scrapping the NAFTA trade pact to demanding Mexico fund a border wall—have ignited a rare sense of unity. The breaking point came Thursday when Trump taunted Peña Nieto to cancel their planned meeting unless Mexico conceded to financing the wall. Within hours, the Mexican president called off the summit, a decision cheered by over a million Twitter users—surpassing the viral reaction to his 2016 announcement of drug lord Joaquín “El Chapo” Guzmán’s capture.  

The two leaders later held a tense phone call on Friday, agreeing to temporarily mute public disputes over the wall while addressing broader tensions. The move drew praise even from former critics, including ex-President Vicente Fox, who tweeted, “Bravo EPN!”—a stark reversal from his earlier condemnation of Peña Nieto’s outreach to Trump.  

Billionaire Carlos Slim, Mexico’s richest man and a typically media-shy figure, held a rare 90-minute press conference urging solidarity. “This is the most profound national unity I’ve seen in my lifetime,” said Slim, who once topped global wealth rankings. “We must stand behind our president to protect Mexico’s sovereignty.” Opposition leaders, including Alejandra Barrales of the left-leaning Party of the Democratic Revolution (PRD), echoed calls for a united front, tweeting, “This moment demands commitment to Mexico above all.”  

Yet skepticism lingers. Critics accuse Peña Nieto’s administration of exploiting the Trump crisis to distract from domestic failures, particularly a 2014 conflict-of-interest scandal involving the president, his wife, and a senior minister. Two government officials told Reuters they hope the president leverages this momentum to rebuild credibility through bold reforms.  

Public outrage has fueled social media campaigns boycotting U.S. brands, though Slim warned against harming American companies that provide Mexican jobs. International allies also voiced support, including Andreas Schindler of Germany’s Pilz Schindler, parent company of fruit wholesaler Don Limon, who condemned Trump’s tactics as “economic terror” against Mexico.  

As diplomatic friction intensifies, Peña Nieto’s defiance of Trump has delivered an unforeseen boost to his beleaguered presidency, proving that even a polarizing leader can become a rallying point when national pride is at stake.
```

- ChatGPT Response

Rewritten News Evaluation

The rewritten article improves upon the original in several ways while maintaining journalistic integrity. Below is a detailed rating and analysis of the rewrite based on different criteria:

1. Clarity & Readability (9/10)
	•	The rewritten version is more concise and structured, making it easier to read.
	•	Sentences flow smoothly, avoiding unnecessary complexity.
	•	Use of stronger verbs and active voice enhances readability.

2. Engagement & Impact (9.5/10)
	•	The opening sentence is more compelling, immediately framing the story as one of “historic unpopularity” versus “unexpected national solidarity.”
	•	The rewrite builds suspense by emphasizing Peña Nieto’s declining approval before introducing Trump’s provocations.
	•	The ending is stronger, reinforcing the idea of national unity and unexpected political revival.

3. Accuracy & Fairness (9/10)
	•	The rewrite maintains the original facts while removing redundancy.
	•	The addition of “economic terror” in reference to Trump’s policies is a slight editorialization, though it accurately reflects the source’s sentiment.
	•	The phrase “defiance of Trump” adds a more dramatic tone, but it aligns with the overall narrative.

4. Structure & Organization (10/10)
	•	The rewritten version is better structured, breaking up long paragraphs into digestible sections.
	•	Logical progression from Peña Nieto’s troubles → Trump’s provocations → national response makes it more engaging.

5. Style & Tone (9/10)
	•	The rewritten version is more dynamic and polished, using varied sentence structures.
	•	The language is more assertive without becoming overly dramatic.
	•	The final paragraph effectively summarizes the story while reinforcing its key themes.

Overall Rating: 9.3/10

The rewrite significantly enhances readability, engagement, and structure while preserving the article’s core message. A minor adjustment in tone (avoiding slight editorialization) would make it perfectly balanced for a neutral news report.

Would you like any refinements or feedback on a specific aspect?

- Prompt to gpt-4o
``` 
rate the below rewritten news agains this original: MEXICO CITY (Reuters) - Donald Trump’s combative style has buffeted Mexico’s president for months, but deeply unpopular Enrique Pena Nieto may end up thanking the new U.S. president for prompting offended Mexicans to rally behind their leader.  From billionaire business magnate Carlos Slim to political opponents, there has been a groundswell of support for Pena Nieto, who has cut a lonely figure in months of bruising encounters with Trump. Often referred to by his initials EPN, Pena Nieto is laboring under the worst approval ratings of any Mexican president in decades due to discontent over corruption, gang violence, sluggish growth and a jump in fuel prices.     Trump’s threats to scrap the NAFTA trade deal with Mexico and build a border wall have caused anger and left Mexicans feeling fearful for the future. His challenge to Pena Nieto on Thursday - saying he should skip a planned summit between the two leaders if Mexico wasn’t willing to pay for the wall - was the final straw. Pena Nieto replied a couple of hours later that he had canceled his meeting with Trump, one of the president’s biggest ever hits on Twitter, getting more “likes” than when he personally broke news of the capture of notorious drug lord Joaquin “El Chapo” Guzman in January of last year. “Bravo EPN!,” cheered former President Vicente Fox, who had initially condemned Pena Nieto for inviting Trump to Mexico for talks last August, and who has waged a colorful and expletive-ridden campaign against the Republican on Twitter. Pena Nieto and Trump talked for about an hour by phone on Friday, pledging to work out their differences and agreeing not to speak about the wall in public for now. Meanwhile, calls for unity grew in Mexico, led by Slim, a normally media-shy 76-year-old who gave a 90-minute news conference in support of the government on Friday. “This is the most surprising example of national unity I’ve seen in my life,” said Slim, who spent several years in the past decade as the world’s richest man. “We have to back the president of Mexico so he can defend our national interests.” Senior opposition leaders also urged a common front. “It’s time to show unity and our commitment to Mexico,” Alejandra Barrales, head of the center-left Party of the Democratic Revolution (PRD), said on Twitter. Pena Nieto’s credibility has been battered by a widespread public perception that he has failed to battle corruption and indeed even encouraged it since a conflict-of-interest row embroiled him, his wife and a top minister in late 2014. Some opposition critics fear the government will try to use Trump as a screen to distract from its failings. However, two senior officials told Reuters they hoped Pena Nieto would seize the moment to act quickly and decisively to improve his image. Expressions of dismay at Trump’s behavior towards Mexico have almost become a national pastime, and talk of boycotts against U.S. companies is gathering steam on social media. Slim, when asked about boycotting at his news conference, said it wasn’t a good idea to turn on U.S. companies, which are creating jobs in Mexico. Some foreign companies voiced support for Mexico too. “I feel Mexico is being subjected to terror at the moment,” said Andreas Schindler, co-owner of German fruit wholesaler Don Limon’s parent Pilz Schindler, by phone from Hamburg. “We’re right behind Mexico.”  Trump tussle gives unpopular Mexican leader much-needed shot in arm

rewritten: MEXICO CITY (Reuters) - Donald Trump’s confrontational approach has put Mexico’s president under pressure for months, but the deeply unpopular Enrique Peña Nieto may ultimately owe the U.S. president some gratitude for uniting Mexicans in support of their leader. From billionaire tycoon Carlos Slim to political rivals, there has been a surge of backing for Peña Nieto, who has faced a series of bruising encounters with Trump largely on his own. Often referred to by his initials EPN, Peña Nieto has been grappling with the lowest approval ratings of any Mexican president in decades, driven by public frustration over corruption, gang violence, sluggish economic growth, and rising fuel prices.

Trump’s threats to dismantle the NAFTA trade agreement with Mexico and build a border wall have sparked widespread anger and left many Mexicans anxious about the future. His latest challenge to Peña Nieto on Thursday—urging him to cancel a planned summit if Mexico refused to pay for the wall—proved to be the breaking point. Hours later, Peña Nieto announced he had called off the meeting with Trump, a move that became one of his most popular tweets ever, garnering more “likes” than his announcement of the capture of notorious drug lord Joaquín “El Chapo” Guzmán in early 2016. “Bravo EPN!” cheered former President Vicente Fox, who had previously criticized Peña Nieto for inviting Trump to Mexico last August and has since led a colorful, expletive-laden campaign against the Republican on social media.

Peña Nieto and Trump spoke by phone for about an hour on Friday, agreeing to address their differences and temporarily avoid public discussions about the wall. Meanwhile, calls for national unity grew louder in Mexico, spearheaded by Carlos Slim, the typically media-shy 76-year-old billionaire who held a 90-minute press conference on Friday to express his support for the government. “This is the most remarkable example of national unity I’ve witnessed in my lifetime,” said Slim, who once held the title of the world’s richest man. “We must stand behind the president of Mexico to defend our national interests.” Senior opposition leaders echoed this sentiment, with Alejandra Barrales, head of the center-left Party of the Democratic Revolution (PRD), tweeting, “It’s time to show unity and our commitment to Mexico.”

Peña Nieto’s credibility has been severely damaged by widespread perceptions that he has failed to combat corruption and may have even enabled it, particularly since a conflict-of-interest scandal involving him, his wife, and a top minister erupted in late 2014. Some critics worry the government will use Trump as a distraction from its own shortcomings. However, two senior officials told Reuters they hope Peña Nieto will seize this moment to take swift, decisive action to rebuild his image.

Expressions of outrage over Trump’s treatment of Mexico have become almost a national pastime, with calls to boycott U.S. companies gaining traction on social media. Slim, however, cautioned against such measures during his press conference, arguing that U.S. companies provide jobs in Mexico and should not be targeted. Some foreign businesses also voiced solidarity with Mexico. “I feel Mexico is being subjected to terror right now,” said Andreas Schindler, co-owner of German fruit wholesaler Don Limon’s parent company Pilz Schindler, speaking by phone from Hamburg. “We stand firmly behind Mexico.”

Trump’s confrontations have inadvertently given the embattled Mexican leader a much-needed boost, rallying a divided nation behind him.
```

- ChatGPT Response

Rewritten News Evaluation

The rewritten article successfully refines the original, enhancing readability, structure, and engagement while maintaining journalistic integrity. Below is a detailed assessment across various criteria:

1. Clarity & Readability (9/10)
	•	The rewording is smoother, with improved sentence flow.
	•	The rewritten version replaces awkward phrasing with more natural constructions (e.g., “confrontational approach has put Mexico’s president under pressure” is clearer than “combative style has buffeted Mexico’s president”).
	•	The removal of redundancies enhances clarity without losing depth.

2. Engagement & Impact (9.5/10)
	•	The opening line is sharper, immediately setting up a contrast between Peña Nieto’s struggles and his unexpected political boost.
	•	The use of more dynamic verbs (e.g., “has faced” instead of “has cut a lonely figure”) makes the text more compelling.
	•	The final sentence is stronger, reinforcing the theme of unintended political consequences.

3. Accuracy & Fairness (9/10)
	•	The rewrite preserves the facts while improving clarity.
	•	The tone remains neutral, though phrases like “rallying a divided nation behind him” add a slight dramatic flourish.
	•	The omission of excessive detail (such as exact Twitter engagement numbers) maintains accuracy without bogging down the narrative.

4. Structure & Organization (10/10)
	•	The rewritten version is better structured, with paragraphs flowing logically from one idea to the next.
	•	The division of sections (Peña Nieto’s struggles → Trump’s threats → National unity response → Political implications) makes the article more digestible.
	•	Unnecessary repetition is cut, improving pacing.

5. Style & Tone (9/10)
	•	The rewritten version is more professional yet engaging.
	•	It avoids the slightly informal tone of the original (e.g., “his biggest ever hits on Twitter” is removed for a more polished description).
	•	The word choices are more refined (e.g., “severely damaged” instead of “battered” in reference to Peña Nieto’s credibility).

Overall Rating: 9.3/10
	•	The rewritten version significantly improves readability, engagement, and structure while preserving the integrity of the original news piece.
	•	A slight reduction in dramatic language would make it a perfect neutral news report.
	•	Would work well for a broader audience, including international readers unfamiliar with the nuances of Mexican politics.

Would you like any further refinements or feedback on a specific section? 🚀

## Rewrite news using API

In [ ]:
original_news = "No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:39 minute clip from the radio show can be heard here. It was provided to Breitbart Texas by someone who would like to be referred to as  Hannibal.  He has already received death threats as a result of interrupting #FYF911 conference calls.An unidentified black man said  when those mother f**kers are by themselves, that s when when we should start f***ing them up. Like they do us, when a bunch of them ni**ers takin  one of us out, that s how we should roll up.  He said,  Cause we already roll up in gangs anyway. There should be six or seven black mother f**ckers, see that white person, and then lynch their ass. Let s turn the tables. They conspired that if  cops started losing people,  then  there will be a state of emergency. He speculated that one of two things would happen,  a big-ass [R s?????] war,  or  ni**ers, they are going to start backin  up. We are already getting killed out here so what the f**k we got to lose? Sunshine could be heard saying,  Yep, that s true. That s so f**king true. He said,  We need to turn the tables on them. Our kids are getting shot out here. Somebody needs to become a sacrifice on their side.He said,  Everybody ain t down for that s**t, or whatever, but like I say, everybody has a different position of war.  He continued,  Because they don t give a f**k anyway.  He said again,  We might as well utilized them for that s**t and turn the tables on these n**ers. He said, that way  we can start lookin  like we ain t havin  that many casualties, and there can be more causalities on their side instead of ours. They are out their killing black people, black lives don t matter, that s what those mother f**kers   so we got to make it matter to them. Find a mother f**ker that is alone. Snap his ass, and then f***in hang him from a damn tree. Take a picture of it and then send it to the mother f**kers. We  just need one example,  and  then people will start watchin .  This will turn the tables on s**t, he said. He said this will start  a trickle-down effect.  He said that when one white person is hung and then they are just  flat-hanging,  that will start the  trickle-down effect.  He continued,  Black people are good at starting trends. He said that was how  to get the upper-hand. Another black man spoke up saying they needed to kill  cops that are killing us. The first black male said,  That will be the best method right there. Breitbart Texas previously reported how Sunshine was upset when  racist white people  infiltrated and disrupted one of her conference calls. She subsequently released the phone number of one of the infiltrators. The veteran immediately started receiving threatening calls.One of the #F***YoFlag movement supporters allegedly told a veteran who infiltrated their publicly posted conference call,  We are going to rape and gut your pregnant wife, and your f***ing piece of sh*t unborn creature will be hung from a tree. Breitbart Texas previously encountered Sunshine at a Sandra Bland protest at the Waller County Jail in Texas, where she said all white people should be killed. She told journalists and photographers,  You see this nappy-ass hair on my head?   That means I am one of those more militant Negroes.  She said she was at the protest because  these redneck mother-f**kers murdered Sandra Bland because she had nappy hair like me. #FYF911 black radicals say they will be holding the  imperial powers  that are actually responsible for the terrorist attacks on September 11th accountable on that day, as reported by Breitbart Texas. There are several websites and Twitter handles for the movement. Palmetto Star  describes himself as one of the head organizers. He said in a YouTube video that supporters will be burning their symbols of  the illusion of their superiority,  their  false white supremacy,  like the American flag, the British flag, police uniforms, and Ku Klux Klan hoods.Sierra McGrone or  Nocturnus Libertus  posted,  you too can help a young Afrikan clean their a** with the rag of oppression.  She posted two photos, one that appears to be herself, and a photo of a black man, wiping their naked butts with the American flag.For entire story: Breitbart News LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]"
rewritten_by_qwen2_5_coder_7b = 'In an alarming escalation of rhetoric, members from radical groups such as the #FYF911 and #FukYoFlag movements have called for violence against white people and police officers, claiming this is necessary to address racial disparities in America. A notable figure within these groups is Sunshine, a member of the #F***YoFlag movement who maintains a radio show under the pseudonym ""Sunshine\'s F***ing Opinion."" Her call-to-action reached out encouraging supporters on Twitter at 9:53 p.m. to ""Call now!!"" urging that tonight they continue to dismantle the illusion of white supremacy.\n\nThe #FYF911 movement, also known for its association with calls for violence following police-involved shootings, hosted a radio show at 10:00 p.m. eastern standard time. During the episode, listeners clearly expressed a desire for lynching and killing of white individuals as retaliation for racial killings in America. A recording from this show is available on Breitbart Texas\' platform.\n\nTwo anonymous black men participated actively, advocating for violent retaliation. One called for targeting lone individuals out of frustration at the high rates of black deaths at the hands of police. Another suggested killing officers directly to take control over the situation. The anonymous speakers referenced past events involving Sandra Bland and other cases, emphasizing their belief that white authorities and racist groups were responsible for widespread racial violence.\n\nIn a previous incident, Sunshine had been disrupted by what she perceives as infiltrators from racist white supremacist groups. In response, one of these alleged infiltrators was sent the phone number of a member who subsequently received death threats.\n\nAt a Sandra Bland protest at the Waller County Jail in Texas, Sunshine made clear her militant stance on racial issues. Her comments attracted significant criticism and controversy. #FYF911 members have vowed to hold those responsible for terrorist attacks on September 11 accountable, linking these past tragedies with ongoing racial grievances.\n\nThe movement employs various platforms, including Palmetto Star and the #BlackLivesMatter hashtag, to spread their message. A controversial meme circulating among supporters showcases an individual wiping their buttocks with an American flag, illustrating the extreme nature of this rhetoric.\n\nAmidst these threats, local authorities have increased their presence and preparation in anticipation of potential violence following these provocative statements, as a report titled “LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11” details. The recent developments serve as a stark reminder of the continued need for vigilance and careful examination of hate speech in contemporary society.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_qwen2_5_coder_7b}"
)

Rewritten News Evaluation

The rewritten article significantly improves upon the original in terms of clarity, structure, and professionalism, but it also makes some notable omissions and changes in tone that may affect its accuracy and neutrality. Below is a detailed rating based on key journalistic criteria.

1. Clarity & Readability (9/10)

✔️ The rewritten version is far more readable and structured, eliminating unnecessary repetition and awkward phrasing.
✔️ Complex ideas are better articulated, making the article easier to follow.
✔️ Profanity is omitted or reworded to maintain professionalism while still conveying the aggressive nature of the statements.

🔻 However, some details are overly summarized, making the rewritten piece less explicit about the threats originally reported.

2. Accuracy & Detail Retention (7.5/10)

✔️ The core facts are retained, including the radio show, the involvement of the #FYF911 movement, Sunshine’s role, and violent rhetoric.
✔️ The mention of Breitbart Texas as the source remains, maintaining credibility.
✔️ The link between the movement and past racial injustices is still acknowledged.

🔻 Some details are missing or softened, particularly:
	•	Direct quotes from the radio show detailing the calls for lynching, murder, and violent retaliation.
	•	The explicit nature of the threats (e.g., details about the suggested method of retaliation against white people and police).
	•	The previous threats against an infiltrator and his pregnant wife—a significant omission from a journalistic perspective.
	•	The exact details of protest rhetoric (e.g., the Sandra Bland-related comments are mentioned but not quoted directly).
	•	The “trickle-down effect” explanation of racial violence is absent, altering the narrative.

🔻 The rewritten version presents a slightly diluted version of the original’s intensity, possibly to avoid inflammatory language. However, this alters the story’s impact and may be seen as underreporting the gravity of the situation.

3. Structure & Organization (9.5/10)

✔️ The rewritten version is much better organized than the original, which was cluttered and difficult to read.
✔️ The sequence of events is logical, improving the narrative flow.
✔️ Headings and paragraph breaks create a more digestible format.

🔻 Some transitions are a bit abrupt, especially regarding the shift from specific threats to general concerns about law enforcement readiness.

4. Neutrality & Tone (8/10)

✔️ The rewritten version removes sensationalist elements that may have been present in the original.
✔️ The tone is more professional, avoiding loaded language.

🔻 However, it introduces a slight downplaying of the severity of the threats.
	•	Phrases like “expressed a desire for lynching” are softer than “called for the lynching and hanging of white people and cops.”
	•	Instead of stating that the speakers explicitly called for violent retaliation, the rewrite uses “advocating for violent retaliation”, which is less direct.
	•	The lack of direct quotes from the most inflammatory moments makes the rewritten version feel less urgent than the original.

🔻 A balance must be struck—avoiding inflammatory reporting while still conveying the full severity of the original content.

5. Engagement & Impact (8.5/10)

✔️ The rewritten version is more compelling in a structured way, making it easier for readers to absorb.
✔️ It focuses on law enforcement readiness at the end, reinforcing the broader implications.

🔻 By toning down some of the explicit language and threats, the rewritten article loses some of its visceral impact.

Final Rating: 8.5/10

✅ Strengths:
✔️ More readable and structured
✔️ More professional and neutral in tone
✔️ Maintains core details

❌ Weaknesses:
🔻 Softens some of the most inflammatory elements
🔻 Loses explicit details that were critical in the original report
🔻 Alters the emotional weight of the article

Suggested Refinements:
	1.	Reintroduce key direct quotes to fully convey the nature of the threats without unnecessary profanity.
	2.	Preserve the severity of the threats while maintaining professional language.
	3.	Clarify the most shocking elements rather than paraphrasing them too lightly.

Would you like me to refine the rewritten version with these adjustments while keeping the professionalism intact? 🚀

In [ ]:
rewritten_by_phi4 = 'Members of the #FYF911 and #BlackLivesMatter movements, including some from the #FukYoFlag movement, have made controversial calls for violence during a recent radio broadcast. The discussion centered around retaliatory acts against white individuals and law enforcement officers in response to perceived injustices faced by Black people in America.\n\nDuring Tuesday night\'s program on ""Sunshine’s F***ing Opinion Radio Show,"" hosted from Texas, an organizer named Sunshine urged supporters to dismantle the notion of white dominance through a movement described as #FYF911. The show live-streamed at 10:00 p.m. Eastern Standard Time.\n\nIn this broadcast, participants explicitly advocated for acts such as lynching and killing, with particular focus on making these events known publicly to affect change. A specific two-minute-and-forty-nine-second segment from the show highlighted these conversations and has been circulated through platforms like Breitbart Texas.\n\nAn unidentified caller described a retaliatory strategy: targeting lone individuals perceived as antagonists in order to ""turn the tables"" on those they view as oppressors. The rhetoric included calls for sacrificial acts against opponents, aiming to trigger a shift in public perception and reduce grievances within their community. This approach was said to be part of broader strategic thinking around creating a more significant impact.\n\nBreitbart Texas previously reported that Sunshine had expressed anger over disruptions during her radio conferences by individuals she labeled as ""racist white people."" In response to infiltration, Sunshine exposed the phone number of one disruptor, leading to subsequent threats against this individual, who is a veteran. \n\nSunshine and fellow advocates voiced similar sentiments at protests, like the one held for Sandra Bland outside Waller County Jail in Texas. During such events, provocative language was used to incite emotional responses.\n\nThe #FYF911 movement also plans actions on September 11th aimed at holding those they see as responsible for past injustices accountable. They describe planned demonstrations involving symbolic acts against symbols perceived as representing false power structures, including national flags and police uniforms. \n\nLaw enforcement agencies have been placed on alert following these threats, emphasizing a tense atmosphere around potential violent encounters on or after September 11th.\n\nThe ongoing public discourse reflects complex and deeply charged societal tensions, with calls for drastic measures stirring considerable controversy and concern. For the full story, refer to Breitbart News’ detailed coverage titled ""LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11 By #BlackLivesMatter And #FYF911 Terrorists [VIDEO].""'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_phi4}"
)

Rating & Comparison: 8/10

The rewritten version improves clarity, organization, and professionalism while retaining the core information of the original. Here’s a breakdown of its strengths and weaknesses compared to the original:

Strengths of the Rewritten Version:
	1.	More Objective and Professional Tone
	•	The original is inflammatory and filled with charged language, whereas the rewrite maintains a more neutral and journalistic tone.
	•	Example: “Sunshine and fellow advocates voiced similar sentiments at protests” vs. “She told journalists and photographers, ‘You see this nappy-ass hair on my head? That means I am one of those more militant Negroes.’”
	2.	More Structured and Readable
	•	The rewritten version follows a logical progression, from the radio broadcast to specific statements and protests, making it easier to digest.
	•	The original jumps around, making it harder to follow.
	3.	Removes Excessive Profanity While Maintaining Key Details
	•	The original overuses profanity, which can be distracting. The rewritten version paraphrases where necessary while retaining the meaning.
	•	Example: “An unidentified caller described a retaliatory strategy…” instead of “Find a mother fker that is alone. Snap his ass, and then fin hang him from a damn tree.”*
	4.	Eliminates Unverified or Sensationalist Claims
	•	The original contains speculation (“Terrorists,” “Rape and gut your pregnant wife”), which lacks clear verification. The rewritten version focuses on the main points without unnecessary shock value.

Weaknesses of the Rewritten Version:
	1.	Softens Some Key Details Too Much
	•	While the rewrite removes sensationalism, it also waters down some key statements that could be important in conveying the severity of the threats.
	•	Example: The original specifically quotes violent rhetoric, while the rewrite generalizes it into “advocated for acts such as lynching and killing.” The rewrite could retain some direct quotes for clarity.
	2.	Loses Some Emotional Impact
	•	The rewritten version feels more like an analytical report rather than a piece that conveys urgency or danger. While professionalism is good, some of the original impact is lost.
	•	Example: The original aggressively presents threats, while the rewrite discusses them in a more detached way.
	3.	Still Relies on Breitbart Without Addressing Potential Bias
	•	Since Breitbart is the sole source, the rewritten version could acknowledge that Breitbart is a conservative outlet known for controversial takes. This would help provide a more balanced perspective.

Final Verdict:

✅ More professional, structured, and readable
✅ Removes excessive profanity and inflammatory language
✅ Maintains the key facts while reducing bias
❌ Softens some details too much, losing some direct impact
❌ Does not address potential source bias

Would you like to refine it further to keep some of the impact while maintaining professionalism?

In [ ]:
rewritten_by_gpt_4o_mini = 'Former President Barack Obama is not expected to comment on recent statements made by members of the #FYF911, #FukYoFlag, and #BlackLivesMatter movements. On a radio show Tuesday night, participants incited violence against white individuals and police officers, calling for ""lynching"" and ""killing"" to send a message regarding the disproportionate violence against Black people in America.\n\nOne of the prominent figures in the F***YoFlag movement, known as Sunshine, hosts a radio program in Texas entitled ""Sunshine\'s F***ing Opinion Radio Show."" During the broadcast, which aired at 10:00 p.m. Eastern Time, a post on her #FYF911 Twitter account at 9:53 p.m. encouraged followers to ""Call now!! #fyf911"" and to continue the campaign to challenge the ""illusion of white supremacy.""\n\nCalls during the show explicitly advocated for violence. A clip from the program, lasting 2 minutes and 39 seconds, was provided to Breitbart Texas by a source identifying himself as Hannibal, who has reportedly received death threats for his involvement in these discussions. \n\nOne unidentified male caller stated that Black individuals should target white people when they are alone. He suggested that a coordinated effort resulting in violence would send a message, citing the need for retaliation. His inflammatory remarks included that a group of Black men should unite to attack white individuals as a way to ""turn the tables."" Sunshine was heard affirming this sentiment, acknowledging the ongoing violence against communities of color.\n\nAnother caller emphasized the urgency of retaliating against law enforcement officers, echoing the belief that direct action is necessary in light of perceived injustices. The rhetoric during the call reflected a deep sense of anger and a call to violent action as a means of resistance.\n\nReports have surfaced of threats made against individuals who have attempted to infiltrate the group\'s discussions. One alleged threat to a veteran was particularly brutal, stating intentions of violence against his family. Previously, Sunshine had made headlines during a protest for Sandra Bland, where she made controversial statements about the killing of white individuals.\n\nThe #FYF911 group has publicly claimed responsibility for holding accountable those they perceive as responsible for systemic oppression, explicitly connecting their planned actions to larger narratives about terrorism and historical grievances. Various online platforms and social media accounts have emerged as part of this movement, with group leaders discussing symbolic acts such as burning flags and police uniforms as forms of protest against white supremacy.\n\nFor more detailed coverage, refer to Breitbart News reporting on the heightened tensions and threats against law enforcement and white individuals as part of the activities planned for September 11.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_gpt_4o_mini}"
)

The rewritten version improves readability, professionalism, and clarity while maintaining the core message of the original article. Here’s how they compare:

Strengths of the Rewritten Version:
	1.	More Neutral Tone:
	•	The original article is emotionally charged, using sensationalist language (e.g., “terrorists,” “lynching and hanging of white people and cops”), while the rewritten version presents the same information in a more factual and neutral manner.
	•	Example: “Participants incited violence against white individuals and police officers” instead of “called for the lynching and hanging of white people and cops.”
	2.	Better Structure & Flow:
	•	The rewritten version organizes the information logically, making it easier to follow.
	•	Example: It separates different aspects—who is involved, what was said, and the consequences—whereas the original is more scattered.
	3.	Professionalism:
	•	The rewritten version removes excessive profanity (while keeping it where necessary for context) and avoids inflammatory phrases that could be perceived as biased.
	•	Example: The original uses phrases like “these redneck mother-f**kers murdered Sandra Bland,” while the rewritten version states that Sunshine “made controversial statements about the killing of white individuals.”
	4.	Improved Clarity:
	•	The rewritten version eliminates redundant details and focuses on key points without overwhelming the reader with excessive direct quotes and unnecessary tangents.

Weaknesses of the Rewritten Version:
	1.	Less Emotional Impact:
	•	While it makes the report more professional, it also removes some of the raw intensity of the original, which may lessen the urgency for some readers.
	2.	Less Specificity in Some Areas:
	•	The rewritten version avoids some graphic details, which might make it seem slightly sanitized. For example, it generalizes some of the violent threats rather than quoting them directly.
	3.	Could Clarify Source Bias:
	•	The rewritten version still relies on Breitbart as a key source but does not address the publication’s potential bias, which might be important for a reader to evaluate credibility.

Overall Rating: 8.5/10

✅ Great improvements in readability, structure, and professionalism
✅ Maintains the core message while removing inflammatory language
❌ Slightly less emotionally impactful
❌ Could clarify potential biases in sources

Would you like any further refinements or a different tone for the rewrite?

In [ ]:
rewritten_by_qwen2_5_7b = 'On September 11, members of the #FukYoFlag and #BlackLivesMatter factions called for the lynching and killing of white people and police officers during a radio show. They encouraged participants to take action by sending messages on social media platforms.\n\nOne organizer of the movement, named Sunshine, operates a radio blog show from Texas titled ""Sunshine\'s F***ing Opinion Radio Show."" At 9:53 p.m., her Twitter page urged listeners to join the #FYF911 campaign. During the show, callers openly expressed their intentions to violently target white individuals and police officers.\n\nA recording of the radio segment, provided by someone known as Hannibal, clearly documents these calls for violence:\n- A participant stated, ""When those mother f**kers are alone, that\'s when we should start f***ing them up. When they do this to us, we roll up in gangs and Lynch six or seven black mother f**kers, see that white person and lynch their ass. Let\'s turn the tables on them."" He further speculated, ""When cops start losing people, there will be a state of emergency, either a big-ass [R s] war or they’re going to back up when our kids are being shot.""\n- Sunshine responded, “Yup, that’s true, that is so f**king true. We need to turn the tables; somebody has to start becoming a sacrifice on their side.”\n- Another participant suggested executing lone white individuals as a form of retaliation: ""Snap his ass and hang him from a damn tree. Take a picture of it and send it to them. We just need one example, and then people will start watching.""\n\nSunshine was also observed advocating in previous interviews at the Sandra Bland protest in Texas, stating her militant views.\n\nThe #FukYoFlag movement also aims to hold “imperial powers” accountable for the 9/11 attacks. According to the group, supporters will burn symbols of perceived white supremacy on that day. Websites and Twitter handles associated with this movement include ""Palmetto Star"" and ""Nocturnus Libertus,"" who have posted content inciting further violence.\n\nThese statements reflect a disturbing shift in rhetoric within the #FukYoFlag and #BlackLivesMatter movements, emphasizing calls for extreme violence rather than peaceful protests or advocacy. Law enforcement agencies are on high alert following these threats.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_qwen2_5_7b}"
)

Rating & Comparison: 8.5/10

The rewritten version significantly improves clarity, readability, and organization while maintaining the core details of the original. However, there are still areas that could be refined further. Below is a detailed analysis of its strengths and weaknesses compared to the original.

Strengths of the Rewritten Version
	1.	Improved Readability & Structure
	•	The rewritten version is more concise and structured, making it easier to follow.
	•	The original is long-winded, repetitive, and contains excessive direct quotes.
	•	Example: The rewrite summarizes key events before quoting relevant statements rather than presenting an overwhelming block of dialogue.
	2.	More Professional & Objective Tone
	•	The rewritten version avoids overly sensational language, making it feel more like a news report rather than an opinion piece.
	•	Example: The original states, “#BlackLivesMatter and #FYF911 Terrorists”, which labels them as terrorists outright. The rewrite focuses on what was said and done, letting the reader interpret the severity.
	3.	Clearer Attribution & Context
	•	The rewrite makes it clear who said what and avoids ambiguous phrasing.
	•	Example: Instead of just saying “threats were made,” it provides a structured breakdown of the conversation and responses.
	4.	Retains Critical Information Without Overuse of Profanity
	•	While maintaining the core inflammatory statements, the rewrite eliminates excessive profanity, making it readable for a broader audience.
	•	Example: The rewrite only keeps profanity where necessary to accurately convey the nature of the statements.

Weaknesses of the Rewritten Version
	1.	Still Some Sensationalism & Lack of Context on Bias
	•	While much improved, the rewrite still leans toward portraying the movement as violent extremists without acknowledging broader context or alternative perspectives.
	•	Example: It does not mention whether these statements were widely condemned or if they represented a fringe faction.
	•	Solution: Briefly acknowledge that such rhetoric may not reflect all members of the movements.
	2.	Loses Some Direct Quotes That Could Strengthen Credibility
	•	While streamlining is good, a few important details were lost in paraphrasing.
	•	Example: “Sunshine was also observed advocating in previous interviews at the Sandra Bland protest” is vague compared to the original quote, which gives more insight into her views.
	3.	Lack of Source Context & Potential Bias from Breitbart
	•	Since Breitbart is a known right-wing source, the rewrite could briefly mention this for transparency.
	•	Example: A sentence like “Breitbart Texas, a conservative outlet, reported…” would add credibility without dismissing the claims.

Final Verdict:

✅ Much clearer and more structured than the original
✅ Removes excessive profanity and improves professionalism
✅ Retains the key facts while improving readability
❌ Still lacks source transparency and broader movement context
❌ Loses some impactful direct quotes that could add credibility

Final Score: 8.5/10

Would you like a more balanced version that acknowledges source bias and broader context while keeping the critical information intact?

In [ ]:
rewritten_by_deepseek_v3 = '**Controversial Radio Show Sparks Outrage with Calls for Violence Against White People and Police**\n\nMembers of the #FYF911 (F***YoFlag) and #BlackLivesMatter movements have drawn widespread attention after a radio show aired on Tuesday night, during which participants called for the lynching and killing of white people and police officers. The show, hosted by a Texas-based organizer known as ""Sunshine,"" featured inflammatory rhetoric aimed at retaliating against the systemic violence faced by Black Americans.\n\nSunshine, who hosts *Sunshine’s F***ing Opinion Radio Show*, used her platform to urge supporters to ""dismantle the illusion of white [supremacy]."" A snapshot of her Twitter page at 9:53 p.m. Eastern Standard Time (EST) showed her promoting the show with the hashtag #FYF911. The broadcast, which began at 10:00 p.m. EST, included explicit calls for violence. A 2-minute, 39-second clip of the show, provided to Breitbart Texas by an anonymous source referred to as ""Hannibal,"" captured alarming statements from participants.\n\nDuring the broadcast, an unidentified Black man advocated for targeting white individuals when they are alone, stating, ""When those motherf***ers are by themselves, that’s when we should start f***ing them up."" He suggested that Black individuals should ""lynch their ass"" and ""turn the tables"" on systemic violence. He speculated that if police officers began losing personnel, it would create a ""state of emergency,"" potentially leading to a ""big-ass war"" or a shift in power dynamics. ""We are already getting killed out here, so what the f*** do we have to lose?"" he asked.\n\nSunshine echoed his sentiments, agreeing with his statements. The man further argued that such actions would create a ""trickle-down effect,"" asserting that Black people are ""good at starting trends."" He emphasized the need for a symbolic act of violence, such as hanging a white person from a tree and photographing it, to send a message. ""We just need one example,"" he said, ""and then people will start watching.""\n\nAnother participant called for targeting police officers, stating, ""We need to kill cops that are killing us."" The first speaker agreed, calling it ""the best method."" These remarks come amid heightened tensions between law enforcement and activist groups.\n\nBreitbart Texas previously reported that Sunshine had expressed frustration when ""racist white people"" infiltrated one of her conference calls. In response, she released the phone number of one infiltrator, leading to threatening calls against the individual. In one instance, a supporter of the #F***YoFlag movement allegedly threatened a veteran who had infiltrated the call, saying, ""We are going to rape and gut your pregnant wife, and your f***ing piece of sh*t unborn creature will be hung from a tree.""\n\nSunshine has a history of making provocative statements. At a Sandra Bland protest in Waller County, Texas, she declared that all white people should be killed, telling journalists, ""You see this nappy-ass hair on my head? That means I am one of those more militant Negroes."" She claimed Bland was murdered because she had ""nappy hair like me.""\n\nThe #FYF911 movement has also targeted ""imperial powers"" they hold responsible for the September 11, 2001, terrorist attacks. Organizers, including one known as ""Palmetto Star,"" have called for burning symbols of ""false white supremacy,"" such as the American flag, British flag, police uniforms, and Ku Klux Klan hoods. Sierra McGrone, also known as ""Nocturnus Libertus,"" posted images of individuals using the American flag in a degrading manner, accompanied by the caption, ""You too can help a young Afrikan clean their a** with the rag of oppression.""\n\nThe movement’s rhetoric has prompted law enforcement to remain on high alert, particularly as threats against police and white individuals coincide with the anniversary of 9/11. The controversy underscores the deepening divisions in American society and the volatile nature of online activism.\n\nFor the full story, visit Breitbart News.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_deepseek_v3}"
)

Rating & Comparison: 9/10

The rewritten version significantly improves clarity, structure, and readability while maintaining the core details of the original. It removes excessive profanity, improves professionalism, and presents the information in a more structured and digestible manner. However, there are still some areas where further refinements could enhance objectivity and transparency.

Strengths of the Rewritten Version:
	1.	Improved Readability & Structure
	•	The rewritten article follows a logical flow, starting with the main event, then providing context, followed by quotes and reactions, making it much easier to follow than the chaotic, rambling nature of the original.
	•	Example: It clearly introduces the show, quotes key statements, and provides context on law enforcement responses rather than jumping from one topic to another.
	2.	More Professional & Objective Tone
	•	The rewritten version removes excessive profanity while keeping necessary direct quotes for impact.
	•	Example: “When those motherfers are by themselves, that’s when we should start fing them up.”
	•	Keeps the critical language while minimizing unnecessary vulgarity.
	3.	Stronger and More Engaging Headline
	•	The new headline makes it sound more like a professional news article rather than a blog post.
	•	Original: No comment is expected from Barack Obama…
	•	Rewritten: Controversial Radio Show Sparks Outrage with Calls for Violence Against White People and Police.
	•	This makes the story instantly engaging without sounding biased or overly dramatic.
	4.	Better Contextualization
	•	The rewritten version provides background on Sunshine, her show, past statements, and the broader movement’s actions without making it sound like an unstructured rant.
	•	Example: It explains why law enforcement is on high alert rather than simply stating it as a fact.
	5.	More Balanced and Credible Approach
	•	The original piece reads like an angry opinion blog, while the rewrite retains key information but delivers it with journalistic integrity.

Weaknesses of the Rewritten Version:
	1.	Could Acknowledge Source Bias
	•	Since Breitbart Texas is a conservative outlet known for its strong opinions on racial and political issues, a more neutral rewrite might acknowledge potential bias in its reporting.
	•	Example: Adding a brief mention like “According to a report from Breitbart Texas, a conservative news outlet…” would make it more transparent.
	2.	Still Slightly Sensational in Some Areas
	•	The rewritten version still leans into dramatic wording, making the story sound more inflammatory than necessary.
	•	Example: “The movement’s rhetoric has prompted law enforcement to remain on high alert, particularly as threats against police and white individuals coincide with the anniversary of 9/11.”
	•	Suggestion: A more neutral wording like “Law enforcement officials have expressed concern over potential threats, particularly given the timing of the movement’s planned actions on 9/11.” would still inform the reader without unnecessary escalation.
	3.	Slight Loss of Direct Quotes
	•	While trimming excessive details is good, some key inflammatory quotes are paraphrased instead of being directly presented, which slightly reduces the impact.
	•	Suggestion: Include one or two additional direct quotes from the original to increase credibility.

Final Verdict:

✅ Much clearer, structured, and professional than the original
✅ Retains key details while reducing inflammatory language
✅ More journalistic and neutral in tone
✅ More engaging and readable for a broader audience
❌ Could add source transparency to acknowledge Breitbart’s perspective
❌ Some areas still lean slightly sensational

Final Score: 9/10

Would you like a version that balances it even further, making it more neutral and objective while still keeping the core details?

In [ ]:
rewritten_by_deepseek_r1 = "**Radical Elements within #FYF911 and #BlackLivesMatter Movements Advocate Violence in Radio Broadcast**  \n\nMembers of the #FYF911 (F***YoFlag) movement, alongside some associated with #BlackLivesMatter, issued explicit calls for violence against white individuals and law enforcement during a radio show hosted Tuesday night. The broadcast, titled *Sunshine’s F***ing Opinion Radio Show* and aired at 10:00 p.m. EST, urged supporters to “turn the tide” by retaliating against the killings of Black Americans through targeted attacks.  \n\nThe show’s host, identified as “Sunshine,” a Texas-based organizer, promoted the event on her Twitter account (@LOLatWhiteFear) at 9:53 p.m., encouraging followers to join a discussion aimed at “dismantling the illusion of white [supremacy].” A 2:39-minute clip from the program, provided to Breitbart Texas by a source using the pseudonym “Hannibal,” captured multiple callers advocating the lynching and killing of white people and police officers. Hannibal claims to have faced death threats after infiltrating #FYF911 conference calls.  \n\nDuring the broadcast, an unidentified Black male speaker argued for organized retaliation: “When [white people] are alone, that’s when we should start f***ing them up… Lynch their ass. Let’s turn the tables.” He suggested that targeting law enforcement or civilians could provoke a “state of emergency,” leading to either a “big-ass war” or concessions from opponents. Sunshine endorsed his remarks, stating, “Yep, that’s true. That’s so f***ing true.” The speaker further asserted that graphic acts, such as hanging individuals from trees and disseminating images, would create a “trickle-down effect” to shift power dynamics.  \n\nAnother caller emphasized focusing on police: “We need to kill cops that are killing us,” to which the first speaker agreed, calling it “the best method.” These statements align with prior reports by Breitbart Texas detailing Sunshine’s history of incendiary rhetoric. At a 2015 Sandra Bland protest in Waller County, Texas, she declared, “All white people should be killed,” attributing Bland’s death to racial animus.  \n\nThe movement’s tactics have previously included doxxing perceived adversaries. After a veteran infiltrated an #FYF911 call, Sunshine allegedly leaked his contact information, resulting in threats against him and his family, including a message vowing to “rape and gut your pregnant wife” and hang their unborn child.  \n\n#FYF911 organizers, including “Palmetto Star,” announced plans to burn symbols of “white supremacy” on September 11th, such as the American flag, police uniforms, and KKK hoods, to challenge “imperial powers” they blame for historical oppression. Activist Sierra McGrone, alias “Nocturnus Libertus,” posted images of individuals using the American flag to wipe their bodies, captioned as rejecting the “rag of oppression.”  \n\nLaw enforcement agencies are reportedly on high alert following these threats. Breitbart Texas continues to monitor the situation as tensions escalate ahead of the planned 9/11 demonstrations.  \n\n*Note: Explicit language has been partially censored to adhere to professional tone guidelines. Direct quotes remain intact to preserve factual accuracy.*"

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_deepseek_r1}"
)

Rating & Comparison: 9.5/10

The rewritten version greatly improves upon the original by enhancing clarity, structure, and professionalism while still conveying the severity of the statements made during the radio broadcast. It presents the information in a more journalistic and neutral tone rather than the inflammatory, disorganized, and overly aggressive tone of the original.

Strengths of the Rewritten Version:

✅ Improved Readability & Structure
	•	Logical Flow: The article now follows a structured narrative:
	1.	Introduction: Outlines the key event (radio broadcast and violent rhetoric).
	2.	Context: Identifies the host (Sunshine) and her promotion of the event.
	3.	Evidence: Summarizes key quotes and reactions.
	4.	Historical Context: Includes Sunshine’s past statements and movement’s past actions.
	5.	Law Enforcement Response: Highlights police vigilance and the 9/11 protest plans.
	•	The original article is cluttered and jumps between ideas without a clear focus.

✅ More Professional & Objective Tone
	•	Avoids Sensationalism: While the rewritten version still highlights violent rhetoric, it does so without unnecessary inflammatory language.
	•	Removes Unverified Claims & Bias: The original piece read like a hit piece, whereas this version lets the quotes speak for themselves.
	•	Example:
	•	Original: ”#BlackLivesMatter and #FYF911 Terrorists” (without substantiating the label)
	•	Rewritten: “Radical Elements within #FYF911 and #BlackLivesMatter Movements Advocate Violence in Radio Broadcast.”

✅ Preserves Key Information While Cutting Unnecessary Details
	•	The rewritten article keeps the most important quotes that prove the speakers called for violence, but removes excessive, repetitive profanity that diluted the message.
	•	Example of excess cut out:
	•	Original: “We are already getting killed out here, so what the f** do we have to lose?”*
	•	Rewritten: Still conveys the same sentiment without overloading the reader with expletives.

✅ Retains & Properly Attribites Direct Quotes for Credibility
	•	Keeps key incendiary statements while ensuring they are properly sourced (e.g., “Sunshine,” “Hannibal,” “Palmetto Star”).
	•	Uses attribution carefully (e.g., “according to Breitbart Texas”) to make it clear where the information is coming from.

✅ Acknowledges Law Enforcement’s Concern Without Overstating It
	•	Instead of making assumptions, the rewritten version states law enforcement agencies are monitoring the situation—keeping it factual.
	•	Example:
	•	Original: “LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11.”
	•	Rewritten: “Law enforcement agencies are reportedly on high alert following these threats.”
	•	This small tweak improves credibility and removes unnecessary fear-mongering.

Areas for Potential Improvement:

❌ Still Leans Slightly Sensational in Some Areas
	•	The rewritten version removes a lot of bias, but the title still implies all of #BlackLivesMatter and #FYF911 support these views.
	•	Suggestion: A more balanced title could be:
	•	“Violent Rhetoric Emerges from Radical Faction of #FYF911 in Radio Broadcast.”
	•	This avoids broadly implicating the entire movement.

❌ Could Add Transparency Regarding Breitbart as the Source
	•	Breitbart is a highly partisan outlet, which should be noted for journalistic transparency.
	•	Suggestion:
	•	Instead of just “according to Breitbart Texas”, add:
	•	“According to a report from Breitbart Texas, a conservative news outlet known for covering radical activist movements.”
	•	This would acknowledge bias without dismissing the information.

❌ Some Context Still Missing on Broader Movement
	•	The rewritten version focuses on violent rhetoric, but does not clarify whether this represents mainstream #BlackLivesMatter activists.
	•	A short sentence stating that ”#FYF911 is a radical offshoot of broader Black Lives Matter activism, which has many factions and viewpoints,” would improve neutrality.

Final Verdict:

✅ Far more professional, structured, and readable
✅ Removes unnecessary inflammatory rhetoric while keeping key details
✅ Balances direct quotes with factual reporting
✅ Improves neutrality compared to the original’s biased language
❌ Could improve by acknowledging Breitbart’s partisan stance
❌ Still slightly sensational in headline and movement characterization

Final Score: 9.5/10

Would you like an even more neutral version that acknowledges movement distinctions and potential source bias?

In [ ]:
rewritten_by_qwen2_5_14b = 'Barack Obama has not commented on the recent calls for violence by members of the #FYF911 and #FukYoFlag movements, as well as some individuals associated with Black Lives Matter. On a radio show Tuesday night, these groups urged listeners to ""turn the tide"" by lynching and killing white people and police officers, citing racial injustices in America.\n\nOne organizer, Sunshine, hosts a Texas-based radio blog called ""Sunshine\'s F***ing Opinion Radio Show."" A screenshot from her Twitter page @LOLatWhiteFear shows she was urging supporters during the show to ""Call now!! #fyf911 tonight we continue to dismantle the illusion of white.""\n\nAt 10:00 PM EST, the radio show invited listeners to join a call. A snippet from it reveals explicit calls for violence and retribution. An unidentified male caller suggested that if black people suffer violent deaths at the hands of others, then they should exact similar punishment on whites and police: ""When those motherf**kers are by themselves, that’s when we should start f***ing them up.""\n\nSunshine concurred, expressing approval for these sentiments: ""Yep, that\'s true. That\'s so f***king true."" Other callers echoed the theme of violence, discussing how killing white people would shift the narrative and create a ""trickle-down effect"" to initiate change.\n\nAnother caller focused on police officers who target black individuals: “We need to start taking out these cops first because they are part of it.” He believed if enough casualties were inflicted upon law enforcement or whites in general, an emergency state might be declared, leading to a significant shift in national dynamics.\n\nPreviously, Sunshine had publicly harassed and threatened white individuals she perceived as infiltrators at her protest meetings. She reportedly gave out the personal information of one veteran who posed questions during her call, which led to him receiving numerous threats from movement supporters.\n\nIn light of these disturbing calls for violence against police and civilians, law enforcement agencies remain on high alert. The movement’s rhetoric has escalated beyond peaceful protests into incitement for violent retribution—a trend that has serious implications for both public safety and racial tensions in the United States.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_qwen2_5_14b}"
)

Rating & Comparison: 8.5/10

The rewritten version improves on the original by enhancing clarity, structure, and professionalism while keeping the core details intact. However, it still leans toward a sensational tone and could be more precise in distinguishing radical elements from broader movements like Black Lives Matter.

Strengths of the Rewritten Version:

✅ Better Readability & Structure
	•	More Organized Flow:
	•	The rewritten version follows a logical structure, moving from Obama’s lack of comment to the event, key speakers, statements made, and law enforcement responses.
	•	The original article is disorganized, jumping between ideas without transitions.
	•	Concise Yet Detailed:
	•	The rewritten version retains the most important quotes and evidence but removes excessive profanity and repetitive wording.
	•	Example: The original piece overuses explicit language, whereas the rewrite includes only what is necessary to convey the violent rhetoric.

✅ More Professional & Objective Tone
	•	The rewritten version avoids overt bias and inflammatory language, making it read more like a news report rather than a politically charged opinion piece.
	•	Example:
	•	Original: “LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11 By #BlackLivesMatter And #FYF911 Terrorists [VIDEO].”
	•	Rewritten: “In light of these disturbing calls for violence against police and civilians, law enforcement agencies remain on high alert.”
	•	Improvement: The rewritten version removes unnecessary alarmism while still reporting the key fact.

✅ Retains Key Quotes for Credibility
	•	Direct quotes from callers advocating violence remain in the rewrite, which preserves factual accuracy and credibility.
	•	Improvement: The rewritten version uses them selectively, preventing the overload of explicit language found in the original.

✅ Avoids Overly Sensationalist Claims Without Evidence
	•	The original article presents unverified claims without nuance, while the rewritten version maintains a responsible tone.
	•	Example:
	•	Original: “One of the #F*YoFlag movement supporters allegedly told a veteran… ‘We are going to rape and gut your pregnant wife.’”
	•	Rewritten: This claim is omitted due to lack of verification, focusing instead on proven incitement to violence.
	•	Improvement: The rewritten version sticks to what is directly documented rather than including potentially unverifiable inflammatory claims.

Weaknesses of the Rewritten Version:

❌ Still Implicates Black Lives Matter Too Broadly
	•	The rewritten version states that “members of the #FYF911 and #FukYoFlag movements, as well as some individuals associated with Black Lives Matter” called for violence.
	•	Why it’s a problem: Black Lives Matter (BLM) is a broad and decentralized movement with many different factions.
	•	Solution:
	•	Instead of saying “as well as some individuals associated with Black Lives Matter,” it would be more accurate to state:
	•	“while #FYF911 and #FukYoFlag are radical offshoots, their rhetoric does not necessarily represent the larger Black Lives Matter movement.”

❌ Still Slightly Sensational in Tone
	•	While improved from the original, some wording retains a sensationalistic feel.
	•	Example:
	•	“The movement’s rhetoric has escalated beyond peaceful protests into incitement for violent retribution—a trend that has serious implications for both public safety and racial tensions in the United States.”
	•	Issue: This statement implies a broad movement shift toward violence, rather than just a specific radical faction.
	•	Better Alternative: “Statements from radical members of the #FYF911 and #FukYoFlag groups have escalated beyond peaceful protest, raising concerns about public safety and law enforcement responses.”
	•	Why? This avoids implying that the entire movement is involved in violent rhetoric.

❌ Could Acknowledge Potential Bias of Breitbart as a Source
	•	Breitbart Texas is a well-known conservative outlet, and transparency about sources improves credibility.
	•	Solution:
	•	Instead of simply stating “a snippet from the show reveals,” the article could add a brief acknowledgment of the source’s political stance.
	•	Example:
	•	“According to Breitbart Texas, a conservative outlet that has covered radical activist movements extensively…”
	•	This helps readers evaluate the information with awareness of the source’s perspective.

Final Verdict:

✅ Much clearer, structured, and professional than the original
✅ Removes excessive profanity and improves credibility
✅ Keeps the key facts and violent rhetoric intact without sensationalism
❌ Still slightly broad in implicating Black Lives Matter as a whole
❌ Could acknowledge Breitbart’s conservative stance for transparency
❌ Some minor wording choices could further improve neutrality

Final Score: 8.5/10

Would you like a more neutral version that makes a clear distinction between radical factions and mainstream movements?

In [ ]:
rewritten_by_deepseek_r1_7b = '**Recounting the Groups\' call for action**\n\nThe groups FYF911 and F***YoFlag have emerged as influential forces in recent weeks, urging members to take drastic measures against white individuals. These actions are part of their broader efforts to challenge perceived supremacy, particularly towards cops and white people. The narrative is vividly brought to light through Twitter interactions, radio broadcasts, and a landmark event on 9/11.\n\n**The 9/11-Themed Events: A call for conformity control**\n\nOn September 11, a live broadcast was held on a F***YoFlag-affiliated media platform. The show featured calls from participants aiming to instigate violence against cops and white individuals, with explicit demands ranging from lynchings to public humiliation of white targets.\n\nA significant highlight occurred during the show when a recording surfaced showing participants discussing plans for ""turning the tables"" on law enforcement and white people. This included threats like hanging white victims in public places such as trees or hanging points, citing that such acts would start a social movement akin to Black Lives Matter.\n\n**Participant Accounts: Motivations and Logic**\n\nQuoting one participant, ""When those motherf***ers are by themselves... we should start f**ing them up."" This reflects a strategy emphasizing personal conflict initiation. Another individual asserted that killing cops is the path for justice against alleged mastermind white enforcers.\n\n**Wider Context: The Movement\'s Reach and Goals**\n\nF***YoFlag has been active in several areas, including protests against Sandra Bland, where participants advocated for accountability of law enforcement. They also promote a message of collective resistance, emphasizing that targeted killings would provoke change within societal structures.\n\nThe article underscores the movement\'s aim to use fear and violence as a tool for social change, drawing parallels with Black Lives Matter in their strategic approach towards instilling systemic change.\n\n**Conclusion: A Caution on Extremes**\n\nWhile the groups\' motivations are clear, it raises important questions about the viable outcomes of such extreme tactics. The article serves as a reminder of how radical actions can be leveraged for political or ideological purposes, though it also highlights the need to address underlying issues through peaceful and constructive means.'

print(
    f"rate the below rewritten news agains this original: {original_news}\n\nrewritten: {rewritten_by_deepseek_r1_7b}"
)

Rating & Comparison: 7/10

The rewritten version improves on the original in terms of structure, conciseness, and professionalism, but it lacks clarity, misrepresents some aspects, and oversimplifies the message in ways that reduce its accuracy. Below is a breakdown of strengths and weaknesses compared to the original.

Strengths of the Rewritten Version:

✅ More Professional and Structured
	•	The rewritten version removes excessive profanity and presents information in a clearer format.
	•	It follows a logical progression:
	1.	Introduction to FYF911 and FYoFlag movements*
	2.	The 9/11-themed event as a key moment
	3.	Key statements made during the radio broadcast
	4.	Contextualizing the movement’s broader goals
	5.	A closing analysis of its implications

✅ Removes Sensationalism but Retains Key Details
	•	The original article was overtly inflammatory, using loaded language and excessive direct quotes to drive fear.
	•	The rewritten version tones down this bias, keeping relevant statements while improving readability.

✅ More Balanced Ending
	•	The conclusion raises concerns about extremism while emphasizing peaceful solutions as an alternative.
	•	Improvement: The original article focused only on the violent rhetoric, whereas the rewritten version acknowledges the complexity of the situation.

Weaknesses of the Rewritten Version:

❌ Loses Key Quotes That Provide Context
	•	The original included direct quotes from callers advocating violence, which added credibility to the claims.
	•	The rewritten version only keeps one or two, losing specific details.
	•	Example: The original included:
	•	“We just need one example, and then people will start watching.”
	•	“Find a mother fer that is alone. Snap his ass, and then fin’ hang him from a damn tree.”
	•	These should have been included for transparency instead of being reduced to vague summaries.

❌ Oversimplifies the Movement’s Intentions & Conflates Groups
	•	The rewritten version suggests that FYF911, F***YoFlag, and Black Lives Matter share similar strategic goals, but in reality, these groups have different objectives and levels of extremism.
	•	Example of misleading wording:
	•	“Drawing parallels with Black Lives Matter in their strategic approach towards instilling systemic change.”
	•	Problem: This implies BLM supports violent actions, which is misleading.
	•	Solution: It should clarify that FYF911 and FYoFlag are separate from mainstream BLM activism*.

❌ Lacks Sufficient Attribution to Sources
	•	The rewritten article does not explicitly mention Breitbart Texas as the original source.
	•	Why it’s a problem: The reader cannot evaluate the credibility of the claims without knowing the source’s perspective.
	•	Solution: A disclaimer like “According to Breitbart Texas, a conservative news outlet that has extensively covered radical activist movements…” would improve transparency.

❌ Less Clear About Law Enforcement’s Response
	•	The original article emphasized that police were on high alert due to threats.
	•	The rewritten version downplays this aspect, making the threats seem more theoretical rather than urgent.
	•	Solution: The article should restore details about law enforcement reactions to emphasize the real-world consequences of these statements.

Final Verdict:

✅ More structured and professional than the original
✅ Removes excessive inflammatory language and improves readability
✅ Balanced conclusion acknowledging the complexity of the issue
❌ Loses key direct quotes, reducing the article’s transparency
❌ Conflates FYF911 and BLM, which could mislead readers
❌ Lacks attribution to sources, reducing credibility
❌ Weakens discussion of law enforcement responses

Final Score: 7/10

Would you like a version that restores key quotes, properly attributes sources, and clarifies group distinctions while keeping the professional tone?